### Loading Libraries

In [1]:
#Init Libraries

#Fitting Libraries
from components import Components
from llama_index.core import Settings

#Additional Core Libraries
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate,Document 
from llama_index.core.node_parser import SentenceSplitter

import pandas as pd

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#initialise components, reranker and retriever
rag_components = Components("Snowflake/snowflake-arctic-embed-s","mixedbread-ai/mxbai-embed-large-v1","llama3-8b-8192")

In [3]:
rag_components.model_name

'llama3-8b-8192'

Setting Contexts


In [4]:
Settings.embed_model = rag_components.get_embedding_model()
Settings.llm = rag_components.get_groq_llm()

Embedding model loaded!


In [5]:
vector_store = ChromaVectorStore(chroma_collection=rag_components.get_db())
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Loading Papers

In [7]:
df = pd.read_csv('../data/MainDataset/data/Context.csv')

In [13]:
context_list = []
for contexts in df['context']:
    context_list.append(Document(text=contexts))

In [14]:
pipeline = IngestionPipeline(
    transformations=[
          SentenceSplitter(chunk_size=512, chunk_overlap=128),
    ],
    vector_store=vector_store,
)

In [15]:
documents = pipeline.run(documents=context_list)

In [16]:
index = VectorStoreIndex(documents, storage_context=storage_context,similarity_top_k=5) #node_postprocessors=[rag_components.get_reranker()]
 

In [17]:
query_engine = index.as_query_engine(streaming=True,similarity_top_k=3)

In [18]:
qa_prompt_template_str = """
Context: {context_str}
Instructions:
- Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
- Utilize the context provided for accurate and specific information.
- Incorporate your preexisting knowledge to enhance the depth and relevance of your response.
- Be concise and to the point.
Question: {query_str}
"""


qa_prompt_template = PromptTemplate(qa_prompt_template_str)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":qa_prompt_template}
)


In [19]:
query_engine.update_prompts(
    {"response_synthesizer:refine_template":PromptTemplate("")}
)


In [21]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [22]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)
    

In [23]:
import time
for i in range(len(question_list)): 
    time.sleep(3)
    print(question_list[i])
    response = query_engine.query(question_list[i])
    response.print_response_stream()
    rowIndex = df_qa.index[i]

    df_qa.at[rowIndex, 'gen_answer'] = response.response_txt
    print('')
    print("--"*100)
    print(f'{i}/{128}')

who introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands? brainly
According to the context, Joseph Lister introduced modern methods of surgery, such as antiseptics, sterilization, and washing hands, with his paper "Antiseptic Principle of the Practice of Surgery" in 1867.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128
what kind of government was chiang kai-shek trying to build in china
According to the context, Chiang Kai-shek was trying to build a Nationalist government in China. He was the leader of the Kuomintang, the nationalist movement in China, and aimed to unify China and fight communism.
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
df_qa

,question,answers,gen_answer
0,"who introduced modern methods of surgery, such...",Joseph Lister,"According to the context, Joseph Lister introd..."
1,what kind of government was chiang kai-shek tr...,Chiang Kai-Shek was trying to build a Communis...,"According to the context, Chiang Kai-shek was ..."
2,president adams,John Quincy Adams was an American statesman wh...,"You're referring to John Quincy Adams, the 6th..."
3,define semitic,A member of any of a number of peoples of anci...,"According to the provided context, a Semite re..."
4,what is wudfhost service,It is a set of Microsoft tools that aid in the...,The WUDFHost.exe is a process that belongs to ...
...,...,...,...
151,"In modern times, what is said about civil diso...","['become utterly debased', 'utterly debased', ...","According to the text, in modern times, the te..."
152,Who noted the different current uses of civil ...,"['Marshall Cohen', 'Marshall Cohen', 'Marshall...","According to the text, Vice President Agnew no..."
153,How has civil disobedience evolved in current ...,['code-word describing the activities of mugge...,"According to the text, civil disobedience has ..."
154,Who gave a negative connotation to civil disob...,"['Vice President Agnew', 'Vice President Agnew...","According to the text, Vice President Agnew ga..."


In [26]:
df_qa.to_csv(f'../data/MainDataset/results/{rag_components.model_name}_RAG_Run1.csv',index=False)